In [ ]:
# -*- coding: utf-8 -*-

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image

import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

from retina.utils import visualize_boxes

MODEL_PATH = '/root/anaconda3/envs/hw3/retinanet-digit-detector/snapshots/resnet152_pascal_04.h5'
#MODEL_PATH = "/root/anaconda3/envs/hw3/retinanet-digit-detector/resnet50_full.h5"

def load_inference_model(model_path=os.path.join('snapshots', 'resnet.h5')):
    model = models.load_model(model_path, backbone_name='resnet152')
    model = models.convert_model(model)
    model.summary()
    return model

def post_process(boxes, original_img, preprocessed_img):
    # post-processing
    h, w, _ = preprocessed_img.shape
    h2, w2, _ = original_img.shape
    boxes[:, :, 0] = boxes[:, :, 0] / w * w2
    boxes[:, :, 2] = boxes[:, :, 2] / w * w2
    boxes[:, :, 1] = boxes[:, :, 1] / h * h2
    boxes[:, :, 3] = boxes[:, :, 3] / h * h2
    return boxes

In [ ]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

In [ ]:
model = load_inference_model(MODEL_PATH)

In [ ]:
# for show one image for predict test
import json
import os
import natsort 

path="/root/anaconda3/envs/hw3/test/"  
path_list=os.listdir(path)
path_list = natsort.natsorted(path_list)

data = []

if __name__ == '__main__':

    image = read_image_bgr("/root/anaconda3/envs/hw3/test/1.png")
    
    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
    
    # preprocess image for network
    image = preprocess_image(draw)
    image, _ = resize_image(draw,416, 470)
    
    
   
    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    print("processing time: ", time.time() - start)
    print(boxes)
    print(scores)
    
    
    boxes = post_process(boxes, draw, image)
    labels = labels[0]
    scores = scores[0]
    boxes = boxes[0]
    
    visualize_boxes(draw, boxes, labels, scores, class_labels=['10', '1', '2', '3', '4', '5', '6', '7', '8', '9'])
    _labels=['10', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    # 5. plot    
    plt.imshow(draw)
    plt.show()
    
    num = sum(i >= 0.5 for i in scores)
    app_box = boxes[:num]
    app_labels = labels[:num]
    app_scores = scores[:num]
    
    app_labels = np.where(app_labels==0,10,app_labels)

    
    temp = app_box.copy()
    for i in range(len(app_box)):
        temp[i,0]=np.copy(app_box[i,1])
        temp[i,1]=np.copy(app_box[i,0])
        temp[i,2]=np.copy(app_box[i,3])
        temp[i,3]=np.copy(app_box[i,2])
        

    print(temp)
    print(app_labels)
    print(app_scores)
    


In [ ]:
import json
import os
import natsort 
import math

path="/root/anaconda3/envs/hw3/test/"  
path_list=os.listdir(path)
path_list = natsort.natsorted(path_list)

data = []


for file_name in path_list:
    img = os.path.join('/root/anaconda3/envs/hw3/test',file_name)
    image = read_image_bgr(img)
    
    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
    
    # preprocess image for network
    image = preprocess_image(image)
    image, _ = resize_image(image, 416, 470)
    
    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    print("processing time: ", time.time() - start)
    
    boxes = post_process(boxes, draw, image)
    labels = labels[0]
    scores = scores[0]
    boxes = boxes[0]
    
    visualize_boxes(draw, boxes, labels, scores, class_labels=['10', '1', '2', '3', '4', '5', '6', '7', '8', '9'])
    _labels = ['10', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    # 5. plot    
    #plt.imshow(draw)
    #plt.show()
    
    num = sum(i >= 0.5 for i in scores)
    app_box = boxes[:num]
    app_labels = labels[:num]
    app_scores = scores[:num]
    
    app_labels = np.where(app_labels==0, 10, app_labels)
    
    print(file_name)

    
    temp = app_box.copy()
    for i in range(len(app_box)):
        temp[i,0]=np.copy(app_box[i,1])
        temp[i,1]=np.copy(app_box[i,0])
        temp[i,2]=np.copy(app_box[i,3])
        temp[i,3]=np.copy(app_box[i,2])
        
    s = np.argsort(temp, axis=0)
    b = s[:,0]
    temp = np.take(temp,b,axis = 0)
    app_labels = np.take(app_labels, b, axis = 0)
    app_scores = np.take(app_scores, b, axis = 0)
    

    temp=np.ceil(temp)

    print(temp)
    print(app_labels)
    print(app_scores)
    
    
    print("--------------------------------------------------")
    print(file_name)
        
    data.append ( {
        'bbox' : temp.tolist(),
        'label' : app_labels.tolist(),
        'score' : app_scores.tolist()
    }
    )
    
with open('0856703_E.json', 'w') as outfile:
    json.dump(data, outfile)   